In [9]:
import numpy as np
import torch
PATH_annotations = "./train/annotations/"
PATH_images = "./train/images/"
CHART_TYPE = {'vertical_bar':0,'horizontal_bar':1,'dot':2,'line':3,'scatter':4}
OBJ_TYPE = {'label_angle00':5,'label_angle45':6,'label_angle90':7,'label_angle-45':8,
            'axis_x':9,'axis_y':10,'tick_point_x':11,'tick_point_y':12,'useless':13}
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import json, os
_,_,annotation_list = next(os.walk(PATH_annotations))
_,_,image_list = next(os.walk(PATH_images))
annotation_list.sort()
image_list.sort()
annotation_list_train = annotation_list[:57500]
annotation_list_test = annotation_list[57500:]
image_list_train = image_list[:57500]
image_list_test = image_list[57500:]

In [11]:
from PIL import Image

for idx in annotation_list:
    string = ""
    # read annotation message
    with open(PATH_annotations+idx) as file:
        this_annotation = json.loads(file.read())
    # get image message
    img = Image.open(PATH_images+idx.split(".")[0]+".jpg")
    img_width, img_height = img.size
    # get xlist and ylist
    x_list = [i['id'] for i in this_annotation['axes']['x-axis']['ticks']]
    y_list = [i['id'] for i in this_annotation['axes']['y-axis']['ticks']]
# class, type, pos_of_whole_pic
    plot_bb = this_annotation['plot-bb']
    x_center = (plot_bb['x0']+plot_bb['width']/2)/img_width
    y_center = (plot_bb['y0']+plot_bb['height']/2)/img_height
    width = plot_bb['width']/img_width
    height = plot_bb['height']/img_height
    classs = CHART_TYPE[this_annotation['chart-type']]
    string = f"{classs} {x_center} {y_center} {width} {height}\n"
# label and angle classify and
    x_axis_classes = []
    y_axis_classes = []
    for i in this_annotation['text']:
        if i["id"] in y_list:
            pos = i['polygon']
            width = pos['x1']-pos['x0']
            height = pos['y2']-pos['y1']
            x_center = (pos['x0']+width/2)/img_width
            y_center = (pos['y1']+height/2)/img_height
            width /= img_width
            height /= img_height
            classs = OBJ_TYPE['label_angle00']
            y_axis_classes.append([x_center,y_center,width,height,classs])
        elif i["id"] in x_list:
            pos = i['polygon']
            xs = np.array((pos['x0'],pos['x1'],pos['x2'],pos['x3']))
            ys = np.array((pos['y0'],pos['y1'],pos['y2'],pos['y3']))
            x_center, y_center = np.sum(xs)/4, np.sum(ys)/4
            width, height = np.max(xs)-np.min(xs), np.max(ys)-np.min(ys)
            poses = np.concatenate((xs[None],ys[None]),axis=0).T
            poses_number = np.array([0,0,0,0])
            for point in poses:
                if point[0]>=x_center and point[1]>=y_center: poses_number[0]+=1
                elif point[0]>=x_center and point[1]<y_center: poses_number[1]+=1
                elif point[0]<x_center and point[1]<y_center: poses_number[2]+=1
                elif point[0]<x_center and point[1]>=y_center: poses_number[3]+=1
            if (poses_number==np.array([2,0,2,0])).all():
                classs=OBJ_TYPE["label_angle-45"]
            elif (poses_number==np.array([0,2,0,2])).all():
                classs=OBJ_TYPE["label_angle45"]
            elif (poses_number==np.array([1,1,1,1])).all():
                if width/height>=1 or len(str(i["text"]))<=2:
                    classs=OBJ_TYPE["label_angle00"]
                elif width/height<1 and len(str(i["text"]))>=3:
                    classs=OBJ_TYPE["label_angle90"]
            else: classs = 14
            x_center /= img_width
            y_center /= img_height
            width /= img_width
            height /= img_height
            x_axis_classes.append([x_center,y_center,width,height,classs])
    x_classs = np.argmax(np.bincount([i[-1] for i in x_axis_classes]))
    for i in y_axis_classes: string += f"{i[-1]} {i[0]} {i[1]} {i[2]} {i[3]}\n"
    for i in x_axis_classes: string += f"{x_classs} {i[0]} {i[1]} {i[2]} {i[3]}\n"
# x_axis and y_axis
    x_axis_right = max([i[0]+i[2]/2 for i in y_axis_classes])
    x_axis_left = min([i[0]-i[2]/2 for i in y_axis_classes])
    y_axis_down = max([i[1]+i[3]/2 for i in y_axis_classes])
    y_axis_top = min([i[1]-i[3]/2 for i in y_axis_classes])
    x_center = (x_axis_left+x_axis_right)/2
    y_center = (y_axis_top+y_axis_down)/2
    width = x_axis_right-x_axis_left
    height = y_axis_down-y_axis_top
    string += f"{OBJ_TYPE['axis_y']} {x_center} {y_center} {width} {height}\n"
    x_axis_right = max([i[0]+i[2]/2 for i in x_axis_classes])
    x_axis_left = min([i[0]-i[2]/2 for i in x_axis_classes])
    y_axis_down = max([i[1]+i[3]/2 for i in x_axis_classes])
    y_axis_top = min([i[1]-i[3]/2 for i in x_axis_classes])
    x_center = (x_axis_left+x_axis_right)/2
    y_center = (y_axis_top+y_axis_down)/2
    width = x_axis_right-x_axis_left
    height = y_axis_down-y_axis_top
    string += f"{OBJ_TYPE['axis_x']} {x_center} {y_center} {width} {height}\n"
# useless
    for i in this_annotation['text']:
        if i['role']=="tick_label": continue
        pos = i['polygon']
        xs = np.array((pos['x0'],pos['x1'],pos['x2'],pos['x3']))
        ys = np.array((pos['y0'],pos['y1'],pos['y2'],pos['y3']))
        x_center, y_center = np.sum(xs)/4, np.sum(ys)/4
        width, height = np.max(xs)-np.min(xs), np.max(ys)-np.min(ys)
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        string += f"{OBJ_TYPE['useless']} {x_center} {y_center} {width} {height}\n"
# tick_point
    for i in this_annotation['axes']['y-axis']['ticks']:
        x_center = i['tick_pt']["x"]/img_width
        y_center = i['tick_pt']["y"]/img_height
        height = 0.033
        if x_center<0.5: width = 2*x_center-0.0005
        else: width = 2*(1-x_center)-0.0005
        string += f"{OBJ_TYPE['tick_point_y']} {x_center} {y_center} {width} {height}\n"
    for i in this_annotation['axes']['x-axis']['ticks']:
        x_center = i['tick_pt']["x"]/img_width
        y_center = i['tick_pt']["y"]/img_height
        width = 0.033
        if y_center>0.5: height = 2*(1-y_center)-0.0005
        else: height = 2*y_center-0.0005
        string += f"{OBJ_TYPE['tick_point_x']} {x_center} {y_center} {width} {height}\n"
    with open(f"./train/labels/{idx.split('.')[0]}.txt",'w') as file:
        file.write(string)


09874dc0e2fe.json
0b507ccf1a99.json
0d90f922e80b.json
133003ec5f4a.json
13a14c58a1d9.json
150bda2d06ba.json
157d1028b7b2.json
15951fbbe751.json
173526fcff49.json
1bd01b1c40d8.json
1bd8fa4655f0.json
2e2bdfc9c7d7.json
2e7492322558.json
347c0f3e475b.json
35f0ec146509.json
36bf2b0f403d.json
3ffc21fad1bd.json
410025e490c3.json
43a6f95994b3.json
45eab7a55c20.json
47130d62b1d1.json
49d4c19d316d.json
4f95cc33ba7f.json
582d866854e0.json
5891714de670.json
5ac507f73eb4.json
5cbcc46a6eed.json
6627e6967a98.json
66347e7f844d.json
6737ada0656e.json
679dc4eee502.json
6881dcba3315.json
6bfddb905b24.json
6e11ee43f6a9.json
725201a26cf4.json
72eb189e7248.json
795a804d0cf2.json
8aba65374b0b.json
8b08dc2d3b10.json
8b6935f7ef04.json
8c5693064aaa.json
996fc1c90e80.json
a79ea0a24df1.json
b3513a46bf1e.json
b6818aa7d7a8.json
b8990ef6a487.json
bc537a717030.json
bdbb0ea41354.json
c2e87653de28.json
c3530c1c7cb5.json
c681d45e7d37.json
ce0becb5f6dd.json
d0f1d33c137f.json
d489b94f79e6.json
d52e57ac6bea.json
d7ec9803eb

In [7]:
with open(f"./train.txt",'w') as file:
    for i in image_list_train:
        file.write(f"{os.getcwd()}\\train\\images\\"+i+"\n")
with open(f"./test.txt",'w') as file:
    for i in image_list_test:
        file.write(f"{os.getcwd()}\\train\\images\\"+i+"\n")
with open(f"./val.txt",'w') as file:
    for i in image_list_test:
        file.write(f"{os.getcwd()}\\train\\images\\"+i+"\n")
with open("./data.yaml",'w') as file:
    file.write(    
"""
train: /kaggle/working/train.txt
val: /kaggle/working/val.txt
test: /kaggle/working/test.txt

nc: 14

names:
  0: vertical_bar
  1: horizontal_bar
  2: dot
  3: line
  4: scatter
  5: label_angle00
  6: label_angle45
  7: label_angle90
  8: label_angle-45
  9: axis_x
  10: axis_y
  11: tick_point_x
  12: tick_point_y
  13: useless
"""
    )